<a href="https://colab.research.google.com/github/MahdiRahbar/NLP_SLU/blob/main/sentiment/WelshSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import os
import re
from collections import defaultdict
fn = os.path.join("..","data","train-v2.tsv")
handle = open(fn, "r")
res = list()
tweets = list()
for i in handle:
    entry = i.strip().split("\t")
    res.append(int(entry[0]))
    tweets.append(entry[1])

In [3]:
# i tried
def reformat_tweets(tweets):
    #get rid of mentions, links, hashtags and numbers
    tweets2 = map(lambda x: re.sub("(@USER|\{URL\}|#\w*|[0-9]+)", "", x).strip(), tweets)
    #lowercase everything
    tweets2 = map(lambda x: x.lower(), tweets2)
    #tokenize
    tweets2 = [t.split(" ") for t in tweets2]
    
    tweets_reformatted = []
    for tweet in tweets2:
        clean_tweet = []
        for word in tweet:
            #keep only face emojis and alpha
            relevant_piece = re.match("[\w\U00010000-\U0010ffff]+", word)
            if relevant_piece:
                clean_tweet.append(relevant_piece.group(0))
        tweets_reformatted.append(clean_tweet)
    return tweets_reformatted

def make_bag_of_words(tweets): # list of list here
    bag_of_words = defaultdict(int)  # key-word, val-count
    for tweet in tweets:
        for word in tweet:
            bag_of_words[word] += 1
    return bag_of_words


In [6]:
from collections import defaultdict

class NaiveBayes():
    def __init__(self, xVec:list, yVec:list, vocabs:defaultdict(int)):
        self.xVec = xVec #assuming xVec = yVec no missing data
        self.yVec = yVec
        self.classes = set(yVec)

        self.vocabs = vocabs
        self.prior = defaultdict(float)
        self.log_like = defaultdict(lambda: defaultdict(float))

    def _sep_by_class(self):
        big_doc = defaultdict(lambda: defaultdict(int))
        for i in range( len(self.xVec)) :
            c = self.yVec[i]
            for w in self.xVec[i]:
                big_doc[c][w] += 1
        return big_doc #big doc contains word count separated by class {c_1: {word1: count, word2:count,etc}, c_2: {word1: count, etc},...}

    def train(self):
        N_all = len(self.xVec)
        big_doc = self._sep_by_class()

        for c in self.classes:
            N_c = sum(1 for i in self.yVec if i == c)
            self.prior[c] = N_c / N_all #calculate prior

            words_in_c = big_doc[c]
            count_all_words_in_c = sum(words_in_c[w] for w in self.vocabs)

            for word in self.vocabs:
                count_word_in_c = words_in_c[word]
                
                # calculate likelihood
                self.log_like[word][c] = math.log(
                    (count_word_in_c+1) / (count_all_words_in_c + len(self.vocabs)) )
    def test(self,tweet):
        bestClass = None
        bestLogPrior = None
        for c in self.classes:
            probability = self.prior[c]
            for w in tweet:
                if w in self.vocabs:
                    probability += self.log_like[w][c]
            if bestLogPrior == None:
                bestLogPrior = probability
                bestClass = c
            elif bestLogPrior < probability:
                bestLogPrior = probability
                bestClass = c
        return bestClass


In [7]:
#divide train-test

train_indexes = range(64000)
tweets_reformatted = reformat_tweets(tweets)

trainX = [tweets_reformatted[i] for i in train_indexes]
trainY = [res[i] for i in train_indexes]
vocabs = make_bag_of_words(trainX)
nb = NaiveBayes(trainX, trainY, vocabs)
nb.train()
nb.test(trainX[0])


TypeError: 'int' object is not iterable

In [233]:

nb.test_bayes(trainX[1])


0

5

In [ ]:
# this should return either 0 (negative sentiment) or 1 (positive sentiment)

def predict_from_scratch(tweet):
  return

In [ ]:
# this should return either 0 (negative sentiment) or 1 (positive sentiment)
def predict_anything_goes(tweet):
  # do something complicated here
  return random.randint(0,1)

In [ ]:
def evaluate():
  total = 0
  correct_from_scratch = 0
  correct_anything_goes = 0
  testfile = open('test.tsv', 'r')
  for line in testfile:
    total += 1
    pieces = line.rstrip("\n").split("\t")
    if predict_from_scratch(pieces[1]) == int(pieces[0]):
      correct_from_scratch += 1
    if predict_anything_goes(pieces[1]) == int(pieces[0]):
      correct_anything_goes += 1
  return (correct_from_scratch/total, correct_anything_goes/total)
  

In [ ]:
evaluate()

(0.5096, 0.4987)